In [ ]:
import cv2
import numpy as np
from tkinter import Tk, Label, Button, filedialog, Canvas, Scale, HORIZONTAL, Frame
from PIL import Image, ImageTk


class ImageProcessingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing Tool")
        self.root.geometry("1300x800")
        self.root.configure(bg="black")

        self.image = None  
        self.processed_image = None  

        # Left Frame
        self.left_frame = Frame(root, bg="black")
        self.left_frame.place(x=50, y=20, width=500, height=600)

        self.left_label = Label(self.left_frame, text="Original Image", font=("Helvetica", 14), bg="black", fg="white")
        self.left_label.pack(pady=10)

        self.left_canvas = Canvas(self.left_frame, width=450, height=450, bg="gray")
        self.left_canvas.pack(pady=10)

        self.upload_button = Button(self.left_frame, text="Upload Image", command=self.upload_image, bg="red", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.upload_button.pack(pady=20)

        # Right Frame
        self.right_frame = Frame(root, bg="black")
        self.right_frame.place(x=750, y=20, width=500, height=600)

        self.right_label = Label(self.right_frame, text="Processed Image", font=("Helvetica", 14), bg="black", fg="white")
        self.right_label.pack(pady=10)

        self.right_canvas = Canvas(self.right_frame, width=450, height=450, bg="gray")
        self.right_canvas.pack(pady=10)

        self.save_button = Button(self.right_frame, text="Save Image", command=self.save_image, bg="red", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.save_button.pack(pady=20)

        # Buttons Frame
        self.buttons_frame = Frame(root, bg="black")
        self.buttons_frame.place(x=580, y=150, width=120, height=400)

        self.grayscale_button = Button(self.buttons_frame, text="Grayscale", command=self.apply_grayscale, bg="blue", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.grayscale_button.pack(pady=15)

        self.blur_button = Button(self.buttons_frame, text="Gaussian Blur", command=self.apply_gaussian_blur, bg="blue", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.blur_button.pack(pady=15)

        self.edge_button = Button(self.buttons_frame, text="Edge Detection", command=self.apply_edge_detection, bg="blue", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.edge_button.pack(pady=15)

        # Brightness and Slider Frame 
        self.brightness_frame = Frame(root, bg="black")
        self.brightness_frame.place(x=50, y=650, width=1200, height=50)  

        self.brightness_button = Button(self.brightness_frame, text="Apply Brightness", command=self.apply_brightness_adjustment, bg="blue", fg="white", font=("Helvetica", 12), width=15, height=2)
        self.brightness_button.pack(side="left", padx=20)

        self.slider = Scale(self.brightness_frame, from_=0, to=100, orient=HORIZONTAL, length=950, bg="black", fg="white", font=("Helvetica", 10))
        self.slider.set(50)
        self.slider.pack(side="left", padx=20)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image = cv2.imread(file_path)
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)  
            self.display_image(self.image, self.left_canvas)

    def display_image(self, img_array, canvas):
        img = Image.fromarray(img_array)
        img.thumbnail((450, 450))  
        tk_image = ImageTk.PhotoImage(img)
        canvas.image = tk_image  
        canvas.create_image(225, 225, image=tk_image, anchor="center")

    def apply_grayscale(self):
        if self.image is None:
            self.left_label.config(text="Please upload an image first!")
            return
        self.processed_image = cv2.cvtColor(self.image, cv2.COLOR_RGB2GRAY)
        self.right_label.config(text="Grayscale")
        self.display_image(self.processed_image, self.right_canvas)

    def apply_gaussian_blur(self):
        if self.image is None:
            self.left_label.config(text="Please upload an image first!")
            return
        k = max(1, self.slider.get() // 10 * 2 + 1)  
        self.processed_image = cv2.GaussianBlur(self.image, (k, k), 0)
        self.right_label.config(text="Gaussian Blur")
        self.display_image(self.processed_image, self.right_canvas)

    def apply_edge_detection(self):
        if self.image is None:
            self.left_label.config(text="Please upload an image first!")
            return
        gray = cv2.cvtColor(self.image, cv2.COLOR_RGB2GRAY)
        param = self.slider.get()
        self.processed_image = cv2.Canny(gray, param, param * 2)
        self.right_label.config(text="Edge Detection")
        self.display_image(self.processed_image, self.right_canvas)

    def apply_brightness_adjustment(self):
        if self.image is None:
            self.left_label.config(text="Please upload an image first!")
            return
        factor = self.slider.get() / 50.0  
        self.processed_image = np.clip(self.image * factor, 0, 255).astype(np.uint8)
        self.right_label.config(text="Brightness Adjustment")
        self.display_image(self.processed_image, self.right_canvas)

    def save_image(self):
        if self.processed_image is None:
            self.right_label.config(text="No processed image to save!")
            return
        file_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
        if file_path:
            if len(self.processed_image.shape) == 2:  
                save_image = self.processed_image
            else:
                save_image = cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR)  
            cv2.imwrite(file_path, save_image)
            self.right_label.config(text="Image saved successfully!")



if __name__ == "__main__":
    root = Tk()
    app = ImageProcessingApp(root)
    root.mainloop()
